# Detector Footprints

In this tutorial we look at the basics of specifying detector footprints. Detector footprints provide templates for filtering which objects are seen in a given pointing. As such we will need to provide both the information about the query point and the survey pointing itself.

Detector footprints are defined by AstroPy regions.

In [ ]:
import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from regions import CircleSkyRegion, RectangleSkyRegion

from lightcurvelynx.astro_utils.detector_footprint import DetectorFootprint

## DetectorFootprint Class

The `DetectorFootprint` class is a wrapper around the AstroPy region that handles data validation, handles transformations, and provides helper functions. Most users will not need to work with the `DetectorFootprint` directly as the wrapping will be handled automatically by the `ObsTable`.

We start by defining a rectangular footprint with a width of 2 degrees and a height of 1 degree.

In [ ]:
center = SkyCoord(ra=0.0, dec=0.0, unit="deg", frame="icrs")
rect_region = RectangleSkyRegion(center=center, width=2.0 * u.deg, height=1.0 * u.deg, angle=0.0)

### Plotting

We can plot the footprint with the `plot` command.

In [ ]:
fp.plot()

However in most cases we will be interested in seeing the footprint in the context of a pointing on the sky. Let's plot the footprint centered on RA=20.0 and dec=10.0 with a 30 degree clockwise rotation.

In [ ]:
fp.plot(center_ra=20.0, center_dec=10.0, rotation=30.0)

The `plot` command can also show points relative to the footprint and whether they are inside or outside the area.

In [ ]:
ra_points = []
dec_points = []
for ra in np.linspace(18, 22, 20):
    for dec in np.linspace(8, 12, 20):
        ra_points.append(ra)
        dec_points.append(dec)
ra_points = np.array(ra_points)
dec_points = np.array(dec_points)

ax = fp.plot(
    center_ra=20.0,
    center_dec=10.0,
    rotation=30.0,
    point_ra=ra_points,
    point_dec=dec_points,
)

### Querying

The code uses the same notation for querying whether a point is within the footprint. Unlike plotting, we might which to query multiple pointings (with different center locations and rotations) at once. The `contains` function can take a vector of center points and rotations. All arrays must be the same length.

In [ ]:
# The query points alternate between (20, 10) and (25, 10))
query_ra = np.array([20.0, 25.0, 20.0, 25.0])
query_dec = np.array([10.0, 10.0, 10.0, 10.0])

# The survey pointings alternate between (20, 10) and (10, 10).
# No rotation is applied.
pointing_ra = np.array([20.0, 20.0, 10.0, 10.0])
pointing_dec = np.array([10.0, 10.0, 10.0, 10.0])

contains = fp.contains(query_ra, query_dec, pointing_ra, pointing_dec)
print(contains)

As we see the first point falls within the footprint, but the remaining points do not.

## Circular Footprints

By default the `ObsTable` class does an initial filtering based on a circular radius. However in some cases we might want to later use a tighter circular radius.  We can do this with the `CircularFootprint` class.

Here we create a circular footprint with a 1 degree radius.

In [ ]:
fp2 = CircularFootprint(1.0)
ax = fp2.plot(
    center_ra=20.0,
    center_dec=10.0,
    point_ra=ra_points,
    point_dec=dec_points,
)